## Generalized Indices ground truth for SSZ-QL GI calculation
The target of this Notebook is to support the development of Generalized Indices calculation in the scope of Generalized Merkle Multiproofs

In [ ]:
from eth2spec.altair.mainnet import get_generalized_index
from eth2spec.utils.ssz.ssz_typing import Container, List, Vector, Bytes32, uint64, byte, bit
from typing import Union as PyUnion

In [ ]:
class FixedNestedContainer(Container):
    Value1: uint64
    Value2: Vector[Bytes32, 32]  # Vector of Bytes32 with fixed length 32


class VariableNestedContainer(Container):
    Value1: uint64
    FieldListUint64: List[uint64, 100]
    NestedListField: List[List[byte, 100], 50]

class VariableTestContainer(Container):
    LeadingField: Bytes32
    FieldListUint64: List[uint64, 2048]
    FieldListContainer: List[FixedNestedContainer, 128]
    FieldListBytes32: List[Bytes32, 100]
    Nested: VariableNestedContainer
    BitlistField: List[bit, 2048]
    NestedListField: List[List[byte, 100], 50]
    TrailingField: Vector[byte, 56]
	


In [36]:
# Container field roots
element_paths = ['Value1', 'Value2']
for element in element_paths:
    gi = get_generalized_index(FixedNestedContainer, element)
    print(f"Generalized index for {element} is {gi}")

# Vector element roots (Value2 is a Vector[Bytes32, 32])
gi_v2_0 = get_generalized_index(FixedNestedContainer, 'Value2', 0)
print(f"Generalized index for Value2[0] is {gi_v2_0}")

gi_v2_31 = get_generalized_index(FixedNestedContainer, 'Value2', 31)
print(f"Generalized index for Value2[31] is {gi_v2_31}")

# List length root (FieldListUint64 is a List[uint64, 100])
gi_list_len = get_generalized_index(VariableNestedContainer, 'FieldListUint64', '__len__')
print(f"Generalized index for len(FieldListUint64) is {gi_list_len}")

# Nested list examples (NestedListField is List[List[byte, 100], 50])
gi_outer_len = get_generalized_index(VariableNestedContainer, 'NestedListField', '__len__')
print(f"Generalized index for len(NestedListField) is {gi_outer_len}")

gi_inner0_len = get_generalized_index(VariableNestedContainer, 'NestedListField', 0, '__len__')
print(f"Generalized index for len(NestedListField[0]) is {gi_inner0_len}")

gi_inner_elem = get_generalized_index(VariableNestedContainer, 'NestedListField', 0, 5)
print(f"Generalized index for NestedListField[0][5] is {gi_inner_elem}")

Generalized index for Value1 is 2
Generalized index for Value2 is 3
Generalized index for Value2[0] is 96
Generalized index for Value2[31] is 127
Generalized index for len(FieldListUint64) is 11
Generalized index for len(NestedListField) is 13
Generalized index for len(NestedListField[0]) is 1537
Generalized index for NestedListField[0][5] is 6144
